In [17]:
from pdf2image import convert_from_path
from pytesseract import image_to_string
import pytesseract
import os 
from PIL import Image
import re
import pandas as pd

In [2]:
POPPLER_PATH = r'C:\Program Files (x86)\poppler-23.05.0\Library\bin'
FILES_PATH = os.path.join(os.getcwd(), '../files')
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [3]:
def convert_pdf_to_img(pdf_file):
    return convert_from_path(pdf_file, poppler_path=POPPLER_PATH)

def convert_image_to_text(file):
    text = image_to_string(file)
    return text

def get_text_from_any_pdf(pdf_file):
    images = convert_pdf_to_img(pdf_file)
    final_text = ""
    for pg, img in enumerate(images):
        final_text += convert_image_to_text(img)
    return final_text

def get_text_from_any_image(image_file):
    image = Image.open(image_file)
    return convert_image_to_text(image)

In [11]:
data = {}
for folder in os.listdir(FILES_PATH):
    for file in os.listdir(os.path.join(FILES_PATH, folder)):
        if file.endswith('.pdf'):
            data[file] = get_text_from_any_pdf(os.path.join(FILES_PATH, folder, file))
        elif file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
            data[file] = get_text_from_any_image(os.path.join(FILES_PATH, folder, file))

In [12]:
data.keys()

dict_keys(['sample2.jpg', 'sample1.pdf', 'sample2.pdf', 'sample3.pdf', 'sample4.pdf', 'sample1.png'])

In [23]:
data['sample2.jpg']

'Application Information\n\nFull Name: Jane Doe\nPhone Number: 555 -0\\00\nHome address: 19. Any Street, Any TOWN, OSA\n\nMailing Address: same as above\n\nEmployment Application\n\nStart Date | End Date Employer Name Position Held Reason for leaving\n1/15/2009 | 6/30/2011 | Any Company Assistant baker relocated\n\n7h Joo [Sholzos| Example Corp.| Baker yerter oyag.\n\n| 8) 1512013) Dreserer AnyCompeny |néad beaver | NiA, current)\n'

In [27]:
# jpg
text = 'Application Information\n\nFull Name: Jane Doe\nPhone Number: 555 -0\\00\nHome address: 19. Any Street, Any TOWN, OSA\n\nMailing Address: same as above\n\nEmployment Application\n\nStart Date | End Date Employer Name Position Held Reason for leaving\n1/15/2009 | 6/30/2011 | Any Company Assistant baker relocated\n\n7h Joo [Sholzos| Example Corp.| Baker yerter oyag.\n\n| 8) 1512013) Dreserer AnyCompeny |néad beaver | NiA, current)\n'

In [36]:
# Extract key-value pairs from text
info_pattern = r"(?P<key>[A-Za-z ]+):\s*(?P<value>[A-Za-z0-9 ,-]+)"
info_matches = re.findall(info_pattern, text)

# Extract employment information
employment_pattern = r"(?P<start_date>\d{1,2}/\d{1,2}/\d{4})\s*\|\s*(?P<end_date>\d{1,2}/\d{1,2}/\d{4})\s*\|\s*(?P<employer>[A-Za-z0-9 ,-]+)\s*\|\s*(?P<position>[A-Za-z0-9 ,-]+)\s*\|\s*(?P<reason>[A-Za-z0-9 ,-]+)"
employment_matches = re.findall(employment_pattern, text)

# Create dataframes from the extracted data
info_df = pd.DataFrame(info_matches, columns=["Key", "Value"])
employment_df = pd.DataFrame(employment_matches, columns=["Start Date", "End Date", "Employer Name", "Position Held", "Reason for leaving"])

# Write dataframes to CSV file
csv_filename = "extracted_data.csv"
with open(csv_filename, "w", newline="") as csvfile:
    info_df.to_csv(csvfile, index=False)
    csvfile.write("\n")
    employment_df.to_csv(csvfile, index=False)

print("Data extracted and saved to", csv_filename)

Data extracted and saved to extracted_data.csv
